In [15]:
import pandas as pd
import numpy as np

In [2]:
cc_data = pd.read_csv('data/cleaned_confirmed_cases.csv')

In [3]:
cc_data['DayOfYear'] =  pd.to_datetime(cc_data['Date']).dt.dayofyear

In [4]:
cc_data = pd.concat([cc_data,pd.get_dummies(cc_data['Country_Region'], prefix='country')],axis=1)

In [5]:
cc_data.head()

,Country_Region,Population,Weight,Date,TargetValue,Med. Age,Density (P/Km²),Urban Pop %,Week,Day,...,country_United States,country_Uruguay,country_Uzbekistan,country_Venezuela,country_Vietnam,country_West Bank and Gaza,country_Western Sahara,country_Yemen,country_Zambia,country_Zimbabwe
0,Afghanistan,27657145,0.058359,2020-01-23,0.0,18.0,60,25.0,4,23,...,0,0,0,0,0,0,0,0,0,0
1,Afghanistan,27657145,0.058359,2020-01-24,0.0,18.0,60,25.0,4,24,...,0,0,0,0,0,0,0,0,0,0
2,Afghanistan,27657145,0.058359,2020-01-25,0.0,18.0,60,25.0,4,25,...,0,0,0,0,0,0,0,0,0,0
3,Afghanistan,27657145,0.058359,2020-01-26,0.0,18.0,60,25.0,4,26,...,0,0,0,0,0,0,0,0,0,0
4,Afghanistan,27657145,0.058359,2020-01-27,0.0,18.0,60,25.0,5,27,...,0,0,0,0,0,0,0,0,0,0


In [6]:
cc_data.drop(['Country_Region', 'first_infection'],axis=1, inplace=True)

In [7]:
feature_cols = list(cc_data)
feature_cols.remove('TargetValue')
feature_cols.remove('Date')

In [8]:
mask_cc_test = (cc_data['Date'] >= '2020-05-20')
mask_cc_train = (cc_data['Date'] < '2020-05-20')

In [9]:
test_cc_df = cc_data.loc[mask_cc_test]
train_cc_df = cc_data.loc[mask_cc_train]

In [10]:
y_train_cc = train_cc_df['TargetValue']
x_train_cc = train_cc_df[feature_cols] 

y_test_cc = test_cc_df['TargetValue']
x_test_cc = test_cc_df[feature_cols]

In [11]:
from sklearn.experimental import enable_hist_gradient_boosting

In [12]:
from sklearn.ensemble import HistGradientBoostingRegressor

In [13]:
est = HistGradientBoostingRegressor()

### Searching for optimal solution

In [16]:
from sklearn.model_selection import RandomizedSearchCV

params_opt = { 'learning_rate': list(np.logspace(np.log10(0.08), np.log10(0.2), base = 10, num = 600)),
             'max_depth': [3, 4, 5],
             'min_samples_leaf':list(range(1,4))}
rand_search = RandomizedSearchCV(est, params_opt, scoring= 'r2', n_iter = 100)
search = rand_search.fit(x_train_cc, y_train_cc)

In [17]:
best_params= search.best_params_
optHGBR = HistGradientBoostingRegressor(learning_rate= best_params['learning_rate'], min_samples_leaf=best_params['min_samples_leaf'],
                      max_depth=best_params['max_depth'])
optHGBR.fit(x_train_cc, y_train_cc)
print(optHGBR.score(x_test_cc, y_test_cc))

0.819739546394733
